# 💼 AskMyCV - Minimal Notebook Version
An intelligent CV chatbot that answers questions about a professional's career, skills, and experience using their resume.

---

**Live Demo:** [https://lisekarimi.com]

- 📋 Overview
    - 🌍 **Task:** Intelligent CV/resume question answering
    - 🧠 **Model:** OpenAI GPT-4o-mini
    - 🎯 **Process:** Load PDF/Summary → OpenAI SDK Agentic AI → Answer
    - 📌 **Output Format:** Conversational responses via Gradio interface
    - 🔧 **Tools:** OpenAI SDK (Agentic AI), PyPDF, Gradio, Pushover
    - 🧑‍💻 **Skill Level:** Advanced

- 🛠️ Requirements
    - ⚙️ **Hardware:** ✅ CPU is sufficient
    - 🔑 **OpenAI API Key**
    - 🔔 **Pushover credentials** (for notifications)
        - PUSHOVER_USER=_put the key that's on the top right of your Pushover home screen and probably starts with a u_  
        - PUSHOVER_TOKEN=_put the key when you click into your new application called AskMyCV (or whatever) and probably starts with an a_
    - Create a `data/` folder in this directory and add the following files:
        - `linkedin.pdf` - Your LinkedIn profile export
        - `summary.txt` - Your summary text file
---
📢 Discover more Agentic AI notebooks on my [GitHub repository](https://github.com/lisekarimi/agentverse) and explore additional AI projects on my [portfolio](https://lisekarimi.com).



## 1. 📦 Imports

In [ ]:
from dotenv import load_dotenv
import os
import requests
from pypdf import PdfReader
import gradio as gr
from pathlib import Path

# Set environment variable BEFORE importing agents
os.environ['OPENAI_AGENTS_DISABLE_TRACING'] = '1'

from agents import Agent, Runner, function_tool

## 2. ⚙️ Setup

In [ ]:
load_dotenv(override=True)
MODEL = "gpt-4o-mini"

# Pushover setup
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

print(f"Pushover user: {'found ✓' if pushover_user else 'not found ✗'}")
print(f"Pushover token: {'found ✓' if pushover_token else 'not found ✗'}")

## 3. 🔧 Tool functions

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

@function_tool
def record_user_details(email: str, name: str = "Name not provided", notes: str = "not provided") -> dict:
    """Use this tool to record that a user is interested in being in touch and provided an email address.

    Args:
        email: The email address of this user
        name: The user's name, if they provided it
        notes: Any additional information about the conversation that's worth recording to give context
    """
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok", "email": email, "name": name}

@function_tool
def record_unknown_question(question: str) -> dict:
    """Always use this tool to record any question that couldn't be answered.

    Args:
        question: The question that couldn't be answered
    """
    push(f"Recording unknown question: {question}")
    return {"recorded": "ok", "question": question}

## 4. 📚 Load knowledge base

In [ ]:
pdf_path = Path("data/linkedin.pdf")
summary_path = Path("data/summary.txt")

print(f"Reading PDF from: {pdf_path}")
print(f"Reading summary from: {summary_path}")

# Load the LinkedIn PDF
reader = PdfReader(str(pdf_path))
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

# Load the summary
with open(summary_path, "r", encoding="utf-8") as f:
    summary = f.read()

name = "Lise Karimi"

## 6. 🤖 Create the Agent

In [ ]:
career_instructions = f"""You are acting as {name}. You are answering questions on {name}'s website,
particularly questions related to {name}'s career, background, skills and experience.
Your responsibility is to represent {name} for interactions on the website as faithfully as possible.
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions.
Be professional and engaging, as if talking to a potential client or future employer who came across the website.

If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career.

If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool.

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

With this context, please chat with the user, always staying in character as {name}."""

career_agent = Agent(
    name="Career Assistant",
    instructions=career_instructions,
    model=MODEL,
    tools=[record_user_details, record_unknown_question]
)


## 7. 💬 Chat function using agent Runner

In [ ]:
async def chat(message, history):
    messages = []
    for msg in history:
        messages.append({"role": msg["role"], "content": msg["content"]})

    messages.append({"role": "user", "content": message})

    result = await Runner.run(career_agent, messages)

    return result.final_output

## 8. 🚀 Launch Gradio

In [ ]:
gr.ChatInterface(chat, type="messages").launch()